# Request generation model

In [17]:
import os
import csv
import json
import pyrosm # type: ignore
import osmnx as ox # type: ignore
import numpy as np # type: ignore
import pandas as pd # type: ignore
import networkx as nx # type: ignore
import geopandas as gpd # type: ignore
import matplotlib.pyplot as plt # type: ignore
from shapely.geometry import Point, LineString # type: ignore
from typing import Tuple

# Set working directory
while os.path.basename(os.getcwd()).lower() != 'carsharingmodelcasestudy':
    os.chdir('..')
assert os.path.basename(os.getcwd()).lower() == 'carsharingmodelcasestudy', os.getcwd()

# Load local functions
from helpers.google_maps_api import get_transit_time
from helpers.google_maps_api import details_from_response

# Pandas settings
pd.options.mode.chained_assignment = None  # Suppress SettingWithCopyWarning
pd.set_option('display.max_columns', None) # Show all columns

In [18]:
# Load data
zones = gpd.read_file('data/sogn-granser-geojson.json') # zones for Public Transport (parishes are used as a proxy for DSB zones)
stations = pd.read_csv('data/20_css_cop_latlng.csv', sep=';', index_col=0)
od_data = pd.read_csv('requests/od_data.csv', sep=';', encoding='utf-8')

# Google Maps directory and file setup
gmaps_data_dir_rel_path = 'data/GoogleMaps/'
gmaps_data_dir = os.path.join(os.getcwd(), gmaps_data_dir_rel_path)
os.makedirs(gmaps_data_dir, exist_ok=True)
gmaps_data_file = os.path.join(gmaps_data_dir, f'gmaps_data.json')

In [19]:
# https://pyrosm.readthedocs.io/en/latest/graphs.html
self = pyrosm.OSM.__init__ # Initialize the OSM object 
osm = pyrosm.OSM("./data/OSM/Copenhagen.osm.pbf")

# Get driving network
drive_net = osm.get_network(network_type="driving", nodes=True)
drive_nodes, drive_edges = drive_net
G_drive = osm.to_graph(drive_nodes, drive_edges, graph_type="networkx")

# Get walking network
walk_net = osm.get_network(network_type="walking", nodes=True)
walk_nodes, walk_edges = walk_net
G_walk = osm.to_graph(walk_nodes, walk_edges, graph_type="networkx")

# Get POIs
pois = osm.get_pois()

In [20]:
def get_stations_gdf():
    stations_gdf = gpd.GeoDataFrame(stations, geometry=gpd.points_from_xy(stations['lng'], stations['lat']), crs="EPSG:4326")
    station_nodes = ox.distance.nearest_nodes(G_drive, X=stations['lng'], Y=stations['lat'])
    stations_gdf['node'] = station_nodes
    return stations_gdf

stations_gdf = get_stations_gdf()

In [21]:
# Define a function to plot stations and POIs
def plot_stations_pois(station_points, pois_filtered):
    # Convert to GeoDataFrame
    pois_filtered_gdf = gpd.GeoDataFrame(pois_filtered, geometry=gpd.points_from_xy(pois_filtered['centroid_lon'], pois_filtered['centroid_lat']), crs="EPSG:4326")
    # Plot the map
    fig, ax = plt.subplots(figsize=(10, 10))
    # Plot the road network as a background
    drive_edges.plot(ax=ax, linewidth=0.5, color="gray", alpha=0.5)
    # Plot stations with a specific color and marker
    station_points.plot(ax=ax, color="blue", marker="o", markersize=50, label="Stations")
    # Plot filtered POIs with another color and marker
    pois_filtered_gdf.plot(ax=ax, color="red", marker="o", markersize=10, label="Filtered POIs")
    # Add legend and title
    plt.legend()
    plt.title("Map of Stations and Filtered POIs in Copenhagen")
    plt.xlabel("Longitude")
    plt.ylabel("Latitude")
    # Show the plot
    plt.show()

In [ ]:
def plot_request(input_point, station=stations_gdf):
    input_point["centroid_lon"] = input_point["origin_lon"]
    input_point["centroid_lat"] = input_point["origin_lat"]
    input_point_od = {
        "origin_lon": input_point["destination_lon"].values[0],
        "origin_lat": input_point["destination_lat"].values[0],
        "centroid_lon": input_point["destination_lon"].values[0],
        "centroid_lat": input_point["destination_lat"].values[0]
    }
    input_point_formatted = pd.concat([input_point, pd.DataFrame([input_point_od])], ignore_index=True)
    plot_stations_pois(station, input_point_formatted)


plot_request(od_data.sample(1))

In [ ]:
def count_zones_crossed(origin: Tuple[float, float], destination: Tuple[float, float], zones_gdf: gpd.GeoDataFrame) -> int:
    # Create Points for origin and destination
    origin_point = Point(origin[1], origin[0])
    destination_point = Point(destination[1], destination[0])
    path_line = LineString([origin_point, destination_point])
    intersecting_zones = zones_gdf[zones_gdf.intersects(path_line)]
    num_zones_crossed = intersecting_zones['sognekode'].nunique()  # Replace 'sognekode' with geodata from price-zones later
    num_zones_crossed = np.ceil(num_zones_crossed/3) #! TEMP: 3 "sogn" equate one zone
    return num_zones_crossed

# Zone Map: https://www.dsb.dk/globalassets/pdf/trafikinformation/231201_dot_zonekort_dec_2023_print_v01.pdf
# Price info: https://webapp.rejseplanen.dk/bin/help.exe/mn?L=vs_rkfc&tpl=prisblad
# Price can be calcuated 
def calculate_price_zones_travelled(zone_count):
    if zone_count <= 2:
        return 20.5  # Fixed price for 1-2 zones
    else:
        return 20.5 + (zone_count - 2) * 6 # linear increase of 6 DKK starting from 3 zones

def price_public_transport(customer_request: pd.Series, verbose = False) -> float:
    """Returns price for by public transport for a given customer request."""
    request_origin = (customer_request["origin_lat"].values[0], customer_request["origin_lon"].values[0])
    request_destination = (customer_request["destination_lat"].values[0], customer_request["destination_lon"].values[0])
    count_zones_crossed_value = count_zones_crossed(request_origin, request_destination, zones)
    price = calculate_price_zones_travelled(count_zones_crossed_value)
    price = price / 7.5 # convert DKK to EUR
    if verbose:
        print(f"count_zones_crossed_value {count_zones_crossed_value}")
        print(f"price {price}")
    return price

test_point = od_data.sample(1)

price_public_transport(test_point, True)

In [24]:
def gmaps_request_already_made(request_input_data_dict):
    # Load the existing data from the file
    try:
        with open(gmaps_data_file, 'r') as f:
            existing_data = json.load(f)
            if not isinstance(existing_data, list):
                raise ValueError("JSON file does not contain a list")
    except Exception as e:
            print(Exception)
            existing_data = []
    # Check file
    if any(d['request_input_data'] == request_input_data_dict for d in existing_data):
        # return the existing data
        existing_data_element = [d for d in existing_data if d['request_input_data'] == request_input_data_dict]
        return existing_data_element
    else:
        return -1

def append_to_gmaps_data_file(file_path, data):
    # Check if the file exists and read its contents
    if os.path.exists(file_path):
        with open(file_path, 'r') as f:
            try:
                existing_data = json.load(f)
                if not isinstance(existing_data, list):
                    raise ValueError("JSON file does not contain a list")
            except json.JSONDecodeError:
                existing_data = []
    else:
        existing_data = []
    # Append the new data to the existing data
    existing_data.append(data)
    # Write the updated data to the file
    with open(file_path, 'w') as f:
        json.dump(existing_data, f)

def google_maps_transit_data(customer_request, verbose = False):
    origin_lat = customer_request['origin_lat'].values[0]
    origin_lon = customer_request['origin_lon'].values[0]
    destination_lat = customer_request['destination_lat'].values[0]
    destination_lon = customer_request['destination_lon'].values[0]
    # the departure time must be in the future, here set to 2025 01 7 12:00
    departure_time = 1736247600 
    request_input_data_dict = {
        "origin_lat": origin_lat,
        "origin_lon": origin_lon,
        "destination_lat": destination_lat,
        "destination_lon": destination_lon,
        "departure_time": departure_time
    }
    gmaps_request_already_made_status = gmaps_request_already_made(request_input_data_dict)
    if gmaps_request_already_made_status == -1:
        print("requesting new google maps data")
        # get data from google maps
        travel_data, raw_data = get_transit_time(origin_lat, origin_lon, destination_lat, destination_lon, departure_time)
        # add input data to raw_data
        raw_data['request_input_data'] = request_input_data_dict
        # Append the data to the local file
        append_to_gmaps_data_file(gmaps_data_file, raw_data)
    else:
        print("google maps already made") if verbose else None
        gmaps_request_already_made_status = gmaps_request_already_made_status[0]
        travel_data = details_from_response(gmaps_request_already_made_status)
    # return the travel data regardless of origin
    return travel_data

## Request Algorithm Implementation

$$u_{ki}=f_u(p_{i,j(k)},T^A_{o(k,i)},T^A_{j(k),d(k)},T^V_{i,j(k)};\beta_k), \qquad (2)$$
$$p_{i,j(k)}=\sum_{in\in\mathcal{L}}P^l\alpha_{i,j(k),l}+T^V_{i,j(k)}P^{CS}, \qquad \forall i \in\mathcal{I}^k$$
$$\mathcal{R}=\big\{
    k\in\mathcal{K} | \exists i \in\mathcal{I}^k, l\in\mathcal{L}:u_{k,i}\geq \max_{m\in\mathcal{M}} U_{m,k}
    \big\}, \qquad (3)$$
$$U_{mk}=\beta^C_kP_{mk}+\beta^V_kT^V{mk}+\beta^A_kT^A_{mk}+\beta^B_kT^B_{mk} \qquad (4.2)$$

In [25]:
def distance_between_two_nodes(origin_node, destination_node, mode="walking", verbose = False) -> float:
    try:
        if mode == "walking":
            G_mode = G_walk
        elif mode == "driving":
            G_mode = G_drive
        else:
            raise ValueError("Invalid mode")
        length = nx.shortest_path_length(G_mode, origin_node, destination_node, weight='length')
        print(f"walking distance: {length}") if verbose else None
        return length
    except Exception as e:
        raise ValueError(f"No Path Found, Exeption: {e}") # alternatively, length = float('inf')

Remember customer utility function:
$$u_{k,i,l} = \beta^C_k(P^l+T^V_{i,j}P^{CS})+\beta^V_kT^V_{i,j}+\beta^A_k(T^A_{o(k),i}+T^A_{j,d(k)}), \qquad \forall i\in\mathcal{I}^k, \forall j\in\mathcal{J}_k,  \forall l\in\mathcal{L}$$

In [26]:
def nearest_N_stations_to_node(origin_node, N):
    """returns touple with station node, station coordinates and distance to the station"""
    # Compute shortest path lengths to all reachable nodes within a reasonable threshold
    distance_threshold = 3000  # 3 km threshold
    lengths = nx.single_source_dijkstra_path_length(G_walk, origin_node, cutoff=distance_threshold, weight='length')
    # Find distances to station nodes only
    nearest_stations = []
    for station_node in stations_gdf['node']:
        if station_node in lengths:
            distance = lengths[station_node]
            nearest_stations.append((station_node, distance))
    # Sort stations by distance and select the nearest N
    nearest_stations.sort(key=lambda x: x[1])
    nearest_n_stations = nearest_stations[:N]
    # Retrieve station coordinates from the station nodes
    result = []
    for station_node, distance in nearest_n_stations:
        station_details = stations_gdf.loc[stations_gdf['node'] == station_node].iloc[0]
        result.append([station_node, (station_details['lat'], station_details['lng']), distance])
    return result

In [27]:
# Calculate the utility of the car-sharing service customer
def calculate_CS_travel_data(customer_request: pd.Series, _station_origin_node, _station_destination_node, verbose = False) -> Tuple[float, float, float]:
    """returns time: OS -> Driving -> SD times"""
    origin_node = customer_request['origin_node'].values[0]
    dest_node = customer_request['destination_node'].values[0]
    walking_dist_from_origin_to_station = distance_between_two_nodes(origin_node, _station_origin_node, "walking", verbose)
    request_driving_dist = distance_between_two_nodes(_station_origin_node, _station_destination_node, "driving", verbose)
    walking_dist_from_station_to_destination = distance_between_two_nodes(_station_destination_node, dest_node, "walking", verbose)
    CS_data_tuple = (walking_dist_from_origin_to_station, request_driving_dist, walking_dist_from_station_to_destination)
    print(f"CS dists: {CS_data_tuple}") if verbose else None
    return CS_data_tuple

In [28]:
def append_origin_destination_nodes_to_row_item(row):
    origin_lat = row['origin_lat']
    origin_lon = row['origin_lon']
    destination_lat = row['destination_lat']
    destination_lon = row['destination_lon']
    origin_node = ox.distance.nearest_nodes(G_walk, X=origin_lon, Y=origin_lat)
    destination_node = ox.distance.nearest_nodes(G_walk, X=destination_lon, Y=destination_lat)
    row['origin_node'] = origin_node
    row['destination_node'] = destination_node
    return row

In [29]:
def preprocess_single_request_row(potential_customer_k, verbose = False, csv_file='./requests/od_travel_data_revised.csv'):
    try:
        with open(csv_file, mode='x', newline='') as file:
            writer = csv.writer(file, delimiter=';')
            writer.writerow(['request_id', 'origin_node', 'destination_node', 'alternative_transportation_data', 
                             'i_stations', 'j_stations', 'cs_travel_data'])
    except FileExistsError:
        pass

    request_id = potential_customer_k.index[0]
    origin_node = potential_customer_k['origin_node'].values[0]
    destination_node = potential_customer_k['destination_node'].values[0]
    
    # For alternative transportation methods
    walking_distance_value = distance_between_two_nodes(origin_node, destination_node, "walking", verbose) # Used for utility calculations
    public_travel_price = price_public_transport(potential_customer_k, verbose)
    google_maps_data_obj = google_maps_transit_data(potential_customer_k, verbose)
    total_duration_minutes, access_time_minutes, transfer_times_minutes_sum = google_maps_data_obj['total_duration_minutes'], google_maps_data_obj['access_time_minutes'], sum(google_maps_data_obj['transfer_times_minutes'])
    alternative_transportation_data = {
        'walking_distance_value': walking_distance_value,
        'public_travel_price': public_travel_price,
        'public_travel_time': total_duration_minutes,
        'public_travel_access_time': access_time_minutes,
        'public_travel_transfer_time': transfer_times_minutes_sum
    }

    # For CS service
    i_stations = nearest_N_stations_to_node(origin_node, 3)
    j_stations = nearest_N_stations_to_node(destination_node, 3)

    cs_travel_data = {}

    for i_idx, i_station in enumerate(i_stations):
        for j_idx, j_station in enumerate(j_stations):
            walk1, drive, walk2 = calculate_CS_travel_data(potential_customer_k,
                                                        i_station[0],
                                                        j_station[0],
                                                        verbose)
            cs_travel_data[(i_idx, j_idx)] = (walk1, drive, walk2)

    with open(csv_file, mode='a', newline='', encoding='utf-8') as file:
        writer = csv.writer(file, delimiter=';')
        writer.writerow([
            request_id, origin_node, destination_node, alternative_transportation_data, i_stations, j_stations,
            cs_travel_data,
        ])

def preprocess_requests(requests, verbose = False):
    for r_idx in range(requests.shape[0]):
        potential_customer_k = requests.iloc[[r_idx]]
        origin_lat, origin_lon = potential_customer_k['origin_lat'].values[0], potential_customer_k['origin_lon'].values[0]
        destination_lat, destination_lon = potential_customer_k['destination_lat'].values[0], potential_customer_k['destination_lon'].values[0]
        print(f"Processing request {r_idx} from {origin_lat}, {origin_lon} to {destination_lat}, {destination_lon}")
        potential_customer_k = potential_customer_k.apply(append_origin_destination_nodes_to_row_item, axis=1)
        preprocess_single_request_row(potential_customer_k, verbose)

In [ ]:
# Preprocess requests and save to file
preprocess_requests(od_data, verbose=False)